# Semantic Retriever

## Vector DB

In [1]:
import chromadb

chroma_client = chromadb.Client()

In [2]:
collection = chroma_client.create_collection(name="my_collection")

In [3]:
collection.add(
    embeddings = [[1,2],[2,1]],
    documents=["This is document 1", "This is document 2"],
    # metadatas=[{"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2"]
)

In [4]:
results = collection.query(
    query_embeddings = [[4,1]],
    n_results=2
)

In [5]:
results

{'ids': [['id2', 'id1']],
 'distances': [[4.0, 10.0]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['This is document 2', 'This is document 1']],
 'uris': None,
 'data': None}

## Embeddings

In [6]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2', 
                                      # cache_folder="/data/base_models"
                                     )

In [7]:
sentences = ["The man is sitting on the tree", "The girl is eating an apple"]
embeddings = embedding_model.encode(sentences)
embeddings

array([[-0.21421541, -0.05434414,  0.1983245 ,  0.10691632,  0.28119728,
         0.32432988,  0.1671172 ,  0.20439652, -0.35982794,  0.11064141,
        -0.08583811,  0.4164258 , -0.36450154,  0.8514863 ,  0.11012026,
         0.92857635, -0.3677544 ,  0.16180651,  0.44544208,  0.44167224,
         0.23353025,  0.02829285,  0.05719928, -0.7051381 ,  0.05321509,
        -0.16388665,  0.05863423,  0.01218422,  0.49752522, -0.15215288,
         0.29764953, -0.0795155 ,  0.7024071 ,  0.42614853, -0.47372615,
        -0.28052774, -0.454651  , -0.31223452,  0.10630041,  0.10367017,
        -0.56606895,  0.16192514,  0.14529678,  0.7097149 , -0.49968305,
         0.2386753 , -0.05783366, -0.46789366, -0.18654984,  0.14146505,
        -0.11448386, -0.13624062,  0.07725897, -0.41289458,  0.56032455,
         0.3067137 ,  0.00458776, -0.04948824,  0.6024603 ,  0.25524247,
         0.00164492, -0.7260487 ,  0.34768093, -0.22945064,  0.00646367,
        -0.28977177, -0.36313847, -0.4536401 ,  0.0

In [8]:
embeddings.shape

(2, 384)

## Indexing and Querying

In [9]:
chunks = ['Llama-2 model comes in the 7B, 13B and 70B parameter variants.', 
         'Llama-2-Chat model is the dialogue fine-tuned model of Llama-2 base model',
         'Llama-2 is safe to use since it limits the generation of violent text']

In [10]:
chunk_embeddings = embedding_model.encode(chunks)

In [11]:
chunk_embeddings.shape

(3, 384)

In [12]:
collection = chroma_client.create_collection(name="rag_llama2")

In [13]:
collection.add(
    embeddings = chunk_embeddings,
    documents=chunks,
    ids=["id1", "id2", "id3"]
)

In [14]:
# query = "different versions of llama-2 model"
query = "what about offensive messages"

results = collection.query(
    query_embeddings = embedding_model.encode(query).tolist(),
    n_results=3
)

In [15]:
results

{'ids': [['id3', 'id2', 'id1']],
 'distances': [[58.672630310058594, 84.29502868652344, 97.0833740234375]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['Llama-2 is safe to use since it limits the generation of violent text',
   'Llama-2-Chat model is the dialogue fine-tuned model of Llama-2 base model',
   'Llama-2 model comes in the 7B, 13B and 70B parameter variants.']],
 'uris': None,
 'data': None}